In [38]:
import os
from sklearn.metrics import log_loss
import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import random,math
seed = 10
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
import eli5

In [39]:
%%time
train_transaction = pd.read_csv('../input/train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv('../input/test_transaction.csv', index_col='TransactionID')

train_identity = pd.read_csv('../input/train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('../input/test_identity.csv', index_col='TransactionID')

# train_pred = pd.read_csv('./simple_xgboost_cv.csv', index_col='TransactionID')
# test_pred = pd.read_csv('./simple_xgboost_pred.csv', index_col='TransactionID')
# test_pred['pred'] = (test_pred['pred1'] + test_pred['pred2'] + test_pred['pred3'] )/3
# del test_pred['pred1'],test_pred['pred2'],test_pred['pred3']

# train_f1 = pd.read_csv('../input/f1_train.csv', index_col='TransactionID')[["f148_amt_counts","f148_card1_counts"]]
# test_f1 = pd.read_csv('../input/f1_test.csv', index_col='TransactionID')[["f148_amt_counts","f148_card1_counts"]]



sample_submission = pd.read_csv('../input/sample_submission.csv', index_col='TransactionID')

CPU times: user 45 s, sys: 2.13 s, total: 47.1 s
Wall time: 47.1 s


In [40]:
train_f5 = pd.read_csv('../input/fd_train3.csv', index_col='TransactionID')
test_f5 = pd.read_csv('../input/fd_test3.csv', index_col='TransactionID')
train_transaction = train_transaction.merge(train_f5, how='left', left_index=True, right_index=True)
test_transaction = test_transaction.merge(test_f5, how='left', left_index=True, right_index=True)

train_f5 = pd.read_csv('../input/fe_train2.csv', index_col='TransactionID')
test_f5 = pd.read_csv('../input/fe_test2.csv', index_col='TransactionID')
train_transaction = train_transaction.merge(train_f5, how='left', left_index=True, right_index=True)
test_transaction = test_transaction.merge(test_f5, how='left', left_index=True, right_index=True)

In [41]:
import pickle


In [42]:
train_transaction['TransactionDT2'] = train_transaction['TransactionDT'].map(lambda x:(x//(3600*24*7)))
test_transaction['TransactionDT2'] = test_transaction['TransactionDT'].map(lambda x:(x//(3600*24*7)))

f = open('emb/index_feature2.pkl','rb')
index_feature = pickle.loads(f.read())
feature_index = {}
for k,v in index_feature.items():
    feature_index[v] = k
    print(k,v)

embeddings_matrix = np.load("emb/spectral3.emb.npy")
print(embeddings_matrix.shape)
embeddings_dict = {}
for i in range((embeddings_matrix.shape[0])):
    embeddings_dict[feature_index[i]] = embeddings_matrix[i,:]

for col in "P_emaildomain".split(","):
    print(col)
    for i in range(24):
        train_transaction["w2v_"+col+"_"+str(i)] = train_transaction[col].fillna(-1).map(lambda x:embeddings_dict.get(col+"_"+str((x)),[0]*24)[i])
        test_transaction["w2v_"+col+"_"+str(i)] = test_transaction[col].fillna(-1).map(lambda x:embeddings_dict.get(col+"_"+str((x)),[0]*24)[i])       
        #         print(train_transaction["w2v_"+col+str(i)])         

for col in "card1,addr1".split(","):
    for i in range(24):
        train_transaction["w2v_"+col+"_"+str(i)] = train_transaction[col].fillna(-1).map(lambda x:embeddings_dict.get(col+"_"+str(int(x)),[0]*24)[i])
        test_transaction["w2v_"+col+"_"+str(i)] = test_transaction[col].fillna(-1).map(lambda x:embeddings_dict.get(col+"_"+str(int(x)),[0]*24)[i])

# for col in "TransactionDT".split(","):
#     print(col)
#     for i in range(24):
#         train_transaction["w2v_"+col+"_"+str(i)] = train_transaction[col].map(lambda x:(x)//(3600*24*7)).map(lambda x:embeddings_dict.get("TransactionDT"+"_"+str(int(x)),[0]*24)[i])
#         test_transaction["w2v_"+col+"_"+str(i)] = test_transaction[col].map(lambda x:(x)//(3600*24*7)).map(lambda x:embeddings_dict.get("TransactionDT"+"_"+str(int(x)),[0]*24)[i])
#         train_transaction["w2v_"+col+"2_"+str(i)] = train_transaction[col].map(lambda x:(x+3600*24*3.5)//(3600*24*7)).map(lambda x:embeddings_dict.get("TransactionDT2"+"_"+str(int(x)),[0]*24)[i])
#         test_transaction["w2v_"+col+"2_"+str(i)] = test_transaction[col].map(lambda x:(x+3600*24*3.5)//(3600*24*7)).map(lambda x:embeddings_dict.get("TransactionDT2"+"_"+str(int(x)),[0]*24)[i])


TransactionDT_0 0
TransactionDT2_0 1
card1_13926 2
addr1_315 3
P_emaildomain_-1 4
amt_68.5 5
card1_2755 6
addr1_325 7
P_emaildomain_gmail.com 8
amt_29.0 9
card1_4663 10
addr1_330 11
P_emaildomain_outlook.com 12
amt_59.0 13
card1_18132 14
addr1_476 15
P_emaildomain_yahoo.com 16
amt_50.0 17
card1_4497 18
addr1_420 19
card1_5937 20
addr1_272 21
amt_49.0 22
card1_12308 23
addr1_126 24
amt_159.0 25
card1_12695 26
P_emaildomain_mail.com 27
amt_422.5 28
card1_2803 29
addr1_337 30
P_emaildomain_anonymous.com 31
amt_15.0 32
card1_17399 33
addr1_204 34
amt_117.0 35
card1_16496 36
addr1_-1 37
amt_75.887 38
card1_4461 39
P_emaildomain_hotmail.com 40
amt_16.495 41
card1_3786 42
P_emaildomain_verizon.net 43
card1_12866 44
P_emaildomain_aol.com 45
amt_40.0 46
card1_11839 47
addr1_226 48
amt_10.5 49
card1_7055 50
amt_57.95 51
card1_1790 52
addr1_170 53
amt_30.0 54
card1_11492 55
amt_100.0 56
addr1_184 57
amt_47.95 58
card1_7005 59
addr1_264 60
amt_186.0 61
card1_7875 62
addr1_299 63
amt_39.0 64
card1_

card1_16972 1081
card1_16998 1082
card1_7178 1083
amt_72.95 1084
card1_16338 1085
card1_17162 1086
card1_8965 1087
card1_5001 1088
card1_2261 1089
card1_11782 1090
amt_234.95 1091
card1_6304 1092
card1_13218 1093
amt_36.5 1094
card1_14080 1095
card1_4100 1096
amt_311.61 1097
card1_11905 1098
amt_83.5 1099
card1_5614 1100
card1_3771 1101
card1_15863 1102
card1_18246 1103
card1_3148 1104
amt_2801.75 1105
card1_9836 1106
card1_3681 1107
amt_470.0 1108
card1_6877 1109
card1_5344 1110
amt_3247.91 1111
card1_6207 1112
card1_10349 1113
amt_644.95 1114
card1_10186 1115
amt_145.0 1116
card1_11509 1117
amt_427.47 1118
card1_7768 1119
amt_352.86 1120
card1_15416 1121
card1_1070 1122
amt_151.0 1123
card1_3553 1124
card1_8908 1125
card1_15688 1126
card1_17321 1127
card1_17055 1128
amt_55.95 1129
amt_170.0 1130
card1_9248 1131
amt_495.15 1132
card1_8160 1133
card1_8411 1134
amt_25.256 1135
card1_1039 1136
card1_10676 1137
amt_703.95 1138
amt_26.0 1139
card1_5973 1140
addr1_157 1141
card1_1857 1142
c

amt_107.47 2631
card1_9072 2632
card1_7574 2633
amt_25.96 2634
card1_5840 2635
amt_6.284 2636
card1_10708 2637
amt_36.37 2638
amt_469.18 2639
card1_17645 2640
card1_4834 2641
P_emaildomain_web.de 2642
amt_51.599 2643
card1_5332 2644
card1_8727 2645
card1_8849 2646
card1_3716 2647
amt_879.35 2648
card1_13488 2649
amt_549.99 2650
card1_1922 2651
amt_8.621 2652
card1_6916 2653
amt_547.95 2654
card1_12477 2655
card1_11143 2656
amt_1861.0 2657
card1_1030 2658
card1_18250 2659
card1_5222 2660
card1_2059 2661
amt_9.062999999999999 2662
amt_44.65 2663
amt_6.707000000000001 2664
amt_9.667 2665
card1_7984 2666
amt_33.473 2667
amt_47.611000000000004 2668
card1_17607 2669
card1_5101 2670
card1_12763 2671
card1_2922 2672
card1_7969 2673
amt_14.863 2674
card1_10270 2675
amt_10.99 2676
card1_4039 2677
card1_14037 2678
card1_7040 2679
amt_201.87 2680
amt_58.36600000000001 2681
card1_6942 2682
card1_16218 2683
amt_1750.0 2684
card1_6934 2685
amt_44.952 2686
amt_38.669000000000004 2687
amt_5.8 2688
amt_

card1_15457 3881
amt_231.95 3882
card1_9938 3883
amt_10.392000000000001 3884
card1_13162 3885
card1_5625 3886
card1_17076 3887
amt_30.936 3888
card1_9715 3889
card1_6630 3890
card1_17045 3891
card1_16963 3892
card1_12962 3893
card1_1508 3894
card1_11686 3895
card1_15879 3896
card1_1335 3897
card1_13834 3898
amt_589.0 3899
amt_60.95 3900
amt_88.81700000000001 3901
amt_422.88 3902
card1_5664 3903
card1_10214 3904
card1_3784 3905
amt_60.662 3906
amt_24.47 3907
card1_6230 3908
card1_4692 3909
amt_7.13 3910
amt_17.673 3911
card1_9550 3912
card1_5996 3913
amt_24.531 3914
amt_404.95 3915
amt_94.47 3916
amt_2594.94 3917
amt_606.72 3918
card1_8233 3919
card1_2968 3920
card1_11657 3921
amt_41.448 3922
card1_13076 3923
card1_1023 3924
amt_48.457 3925
card1_7862 3926
card1_6921 3927
card1_4383 3928
card1_11967 3929
P_emaildomain_sc.rr.com 3930
amt_50.3 3931
card1_5853 3932
card1_2807 3933
card1_9421 3934
card1_6051 3935
P_emaildomain_ptd.net 3936
card1_10652 3937
card1_13199 3938
card1_13514 3939


amt_64.77 5067
card1_8321 5068
addr1_218 5069
amt_34.137 5070
card1_13680 5071
card1_10106 5072
card1_11981 5073
card1_11693 5074
amt_16.979 5075
card1_7349 5076
card1_1929 5077
amt_425.84 5078
amt_40.965 5079
card1_5950 5080
card1_1079 5081
amt_71.296 5082
card1_13432 5083
card1_13155 5084
card1_2373 5085
amt_35.708 5086
card1_2049 5087
addr1_127 5088
amt_701.95 5089
card1_13341 5090
card1_14089 5091
amt_1958.0 5092
card1_6619 5093
card1_10503 5094
amt_67.278 5095
card1_5354 5096
card1_5758 5097
card1_2251 5098
card1_14326 5099
card1_1954 5100
amt_465.0 5101
card1_14743 5102
amt_365.93 5103
card1_17001 5104
card1_14926 5105
card1_7997 5106
card1_10631 5107
addr1_403 5108
amt_121.95 5109
card1_11140 5110
amt_607.78 5111
card1_17398 5112
amt_2745.65 5113
card1_12364 5114
card1_9613 5115
card1_10358 5116
card1_15106 5117
amt_837.09 5118
card1_2841 5119
amt_18.8 5120
card1_2632 5121
card1_11574 5122
card1_1778 5123
card1_13814 5124
card1_15236 5125
card1_12459 5126
amt_1307.07 5127
amt_16

...

amt_337.74 42609
card1_2227 42610
amt_84.447 42611
amt_264.87 42612
amt_575.55 42613
amt_209.84 42614
amt_57.812 42615
card1_7606 42616
amt_153.639 42617
amt_2383.24 42618
amt_58.73 42619
amt_395.45 42620
amt_49.812 42621
card1_15434 42622
amt_2323.92 42623
amt_9.295 42624
amt_500.45 42625
amt_1186.53 42626
amt_206.5 42627
amt_466.65 42628
card1_16787 42629
amt_174.726 42630
card1_14659 42631
amt_592.37 42632
amt_3361.17 42633
amt_457.19 42634
amt_1693.35 42635
amt_267.02 42636
amt_685.49 42637
amt_91.728 42638
amt_1174.06 42639
amt_825.15 42640
amt_169.31599999999997 42641
amt_99.65899999999999 42642
amt_30.388 42643
amt_24.088 42644
amt_423.48 42645
amt_393.89 42646
amt_470.32 42647
amt_378.18 42648
amt_36.357 42649
amt_2774.89 42650
amt_179.96099999999998 42651
amt_367.41 42652
amt_559.94 42653
amt_1956.39 42654
card1_2064 42655
amt_6609.95 42656
amt_1085.95 42657
amt_994.93 42658
card1_14921 42659
amt_73.93 42660
amt_6079.95 42661
amt_395.43 42662
amt_515.86 42663
amt_6279.95 42664

amt_800.94 44033
amt_103.42 44034
amt_303.26 44035
amt_703.47 44036
card1_3433 44037
card1_5087 44038
amt_96.09299999999999 44039
amt_377.15 44040
amt_1070.0 44041
amt_106.655 44042
amt_24.463 44043
amt_236.69799999999998 44044
amt_651.27 44045
amt_1714.78 44046
amt_424.59 44047
amt_1275.37 44048
amt_372.24 44049
amt_66.645 44050
amt_1522.77 44051
amt_458.29 44052
amt_14.763 44053
amt_37.953 44054
amt_39.562 44055
amt_1948.95 44056
amt_50.479 44057
amt_387.21 44058
card1_9862 44059
amt_1295.25 44060
amt_71.944 44061
amt_2320.38 44062
amt_2350.27 44063
card1_15168 44064
amt_402.67 44065
card1_13633 44066
amt_125.696 44067
amt_16.517 44068
amt_5.46 44069
amt_745.55 44070
amt_25.846999999999998 44071
amt_474.0 44072
amt_724.31 44073
amt_858.47 44074
amt_383.94 44075
card1_7891 44076
amt_1200.5 44077
amt_11.562000000000001 44078
amt_60.489 44079
amt_2971.51 44080
amt_54.28 44081
amt_546.11 44082
amt_139.19 44083
amt_3520.95 44084
amt_5.781000000000001 44085
amt_19.805999999999997 44086
amt

amt_200.917 44975
amt_350.13 44976
amt_1225.61 44977
amt_476.16 44978
amt_284.43 44979
amt_1962.71 44980
amt_231.165 44981
amt_413.42 44982
amt_1440.59 44983
amt_581.81 44984
amt_359.23 44985
amt_891.01 44986
amt_406.18 44987
amt_237.21 44988
amt_900.13 44989
card1_3359 44990
amt_307.39 44991
amt_806.62 44992
amt_1657.95 44993
card1_12053 44994
amt_193.49 44995
card1_13132 44996
amt_280.3 44997
card1_14113 44998
card1_3030 44999
amt_1606.15 45000
amt_421.75 45001
amt_220.713 45002
amt_76.742 45003
amt_1403.59 45004
amt_2454.95 45005
amt_510.42 45006
amt_1073.31 45007
amt_2278.01 45008
amt_360.98 45009
amt_1238.1 45010
amt_840.39 45011
amt_818.46 45012
addr1_354 45013
amt_691.34 45014
card1_6618 45015
amt_258.48 45016
card1_11594 45017
card1_17857 45018
card1_9526 45019
card1_12103 45020
amt_64.211 45021
amt_606.42 45022
amt_46.756 45023
amt_138.391 45024
amt_1237.1 45025
card1_17471 45026
amt_503.19 45027
card1_17513 45028
amt_1447.51 45029
amt_279.181 45030
card1_9923 45031
card1_4633

card1_14336 46623
card1_12416 46624
amt_2492.45 46625
amt_371.71 46626
amt_101.749 46627
addr1_169 46628
amt_199.62599999999998 46629
card1_2881 46630
addr1_460 46631
amt_37.779 46632
card1_18381 46633
card1_6731 46634
card1_4382 46635
amt_220.22099999999998 46636
card1_9745 46637
amt_178.45 46638
card1_8219 46639
amt_545.25 46640
amt_46.633 46641
card1_1964 46642
card1_8130 46643
card1_9694 46644
amt_1725.0 46645
amt_1875.0 46646
amt_483.4 46647
amt_61.571999999999996 46648
card1_18085 46649
amt_31.8 46650
card1_17888 46651
amt_109.65 46652
card1_16846 46653
amt_407.07 46654
card1_4020 46655
amt_34.181 46656
amt_256.33 46657
amt_148.41299999999998 46658
amt_7.285 46659
amt_281.27099999999996 46660
amt_314.73 46661
card1_12838 46662
card1_8877 46663
amt_379.08 46664
amt_2718.32 46665
amt_757.17 46666
amt_2679.58 46667
card1_12989 46668
card1_4000 46669
amt_366.09 46670
amt_102.395 46671
card1_3913 46672
amt_167.041 46673
card1_16357 46674
amt_545.32 46675
amt_619.22 46676
amt_188.928 4

P_emaildomain


In [43]:
train_transaction['hour'] = train_transaction['TransactionDT'].map(lambda x:(x//3600)%24)
test_transaction['hour'] = test_transaction['TransactionDT'].map(lambda x:(x//3600)%24)
# train_transaction['hour2'] = train_transaction['TransactionDT'].map(lambda x:(x//(3600*24)))
# test_transaction['hour2'] = test_transaction['TransactionDT'].map(lambda x:(x//(3600 * 24)))
# train_transaction['monthday'] = train_transaction['TransactionDT'].map(lambda x:(x//(3600 * 24))%30)
# test_transaction['monthday'] = test_transaction['TransactionDT'].map(lambda x:(x//(3600 * 24))%30)
train_transaction['weekday'] = train_transaction['TransactionDT'].map(lambda x:(x//(3600 * 24))%7)
test_transaction['weekday'] = test_transaction['TransactionDT'].map(lambda x:(x//(3600 * 24))%7)
train_transaction['TransactionAmt_decimal'] = ((train_transaction['TransactionAmt'] - train_transaction['TransactionAmt'].astype(int)) * 1000).astype(int)
test_transaction['TransactionAmt_decimal'] = ((test_transaction['TransactionAmt'] - test_transaction['TransactionAmt'].astype(int)) * 1000).astype(int)

In [44]:
from collections import Counter
from math import log2,log10
def entropy(pr):
    total = len(pr)
    pr = Counter(pr)
    log2 = math.log2
    ent = 0
    for i in pr:
        p = float(pr[i]) / total
        ent += (-p) * log2(p)
    return ent

In [45]:
train_test = train_transaction.append(test_transaction)
for col in [["card1","addr1"],["card2","addr1"],["card1","card2"]]:
    train_transaction["_".join(col)] = train_transaction[col[0]].fillna(-1).map(str) + "_" + train_transaction[col[1]].fillna(-1).map(str)
    test_transaction["_".join(col)] = test_transaction[col[0]].fillna(-1).map(str) + "_" + test_transaction[col[1]].fillna(-1).map(str)
    
train_test = train_transaction.append(test_transaction)
# for col in [["card1","addr1"],["card2","addr1"],["card1","card2"]]:
#     col_count = train_test.groupby("_".join(col))['TransactionDT'].count()
#     train_transaction["_".join(col)+'_count'] = train_transaction["_".join(col)].map(col_count)
#     test_transaction["_".join(col)+'_count'] = test_transaction["_".join(col)].map(col_count)
#     del train_transaction["_".join(col)],test_transaction["_".join(col)]


for col in ["card1","card1_card2","card2_addr1"]:
    col_count = train_test.groupby(col)['TransactionAmt'].mean()
    train_transaction[col+'_amtcount'] = train_transaction[col].map(col_count)
    test_transaction[col+'_amtcount'] = test_transaction[col].map(col_count)
#     train_transaction[col+'_amt2count'] = (train_transaction['TransactionAmt'] - train_transaction[col].map(col_count))/train_transaction[col].map(col_count)
#     test_transaction[col+'_amt2count'] = (test_transaction['TransactionAmt'] - test_transaction[col].map(col_count))/test_transaction[col].map(col_count)

for col in [["card2","addr1"],["card1","card2"],["card1","addr1"]]:
    col_count = train_transaction.groupby(col[0])[col[1]].apply(entropy)
    train_transaction['en_' + col[0] +col[1] + '_count'] = train_transaction[col[0]].map(col_count)
    col_count = test_transaction.groupby(col[0])[col[1]].apply(entropy)
    test_transaction['en_' + col[0] +col[1] + '_count'] = test_transaction[col[0]].map(col_count)  
    del train_transaction["_".join(col)],test_transaction["_".join(col)]
    
for col in "ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,M1,M2,M3,M4,M5,M6,M7,M8,M9".split(","):
    col_count = train_test.groupby(col)['TransactionDT'].count()
    train_transaction[col+'_count'] = train_transaction[col].map(col_count)
    test_transaction[col+'_count'] = test_transaction[col].map(col_count)
    
    
#     s = set(train_transaction[col].tolist())
#     print(col,test_transaction[col].map(lambda x:0 if x in s else 1).sum())

# for col in "card1,card2,card3,card5,addr1,addr2".split(","):
#     bin = 5
#     train_transaction[col+'_rankcount'] = 0
#     test_transaction[col+'_rankcount'] = 0
#     for i in range(bin):
#         index = train_transaction.iloc[train_transaction.shape[0] * i//5:(train_transaction.shape[0] * (i+1))//5,:].index
#         col_count = train_transaction.loc[index,:].groupby(col)['TransactionDT'].count().sort_values()
#         col_count[:] = np.array(range(col_count.shape[0]))/col_count.shape[0]
#         train_transaction.loc[index,col+'_rankcount'] = train_transaction.loc[index,col].map(col_count)
# #         print(train_transaction.loc[index,col+'_rankcount'])
#     for i in range(bin):
#         index = test_transaction.iloc[test_transaction.shape[0] * i//5:(test_transaction.shape[0] * (i+1))//5,:].index
#         col_count = test_transaction.loc[index,:].groupby(col)['TransactionDT'].count().sort_values()
#         col_count[:] = np.array(range(col_count.shape[0]))/col_count.shape[0]
#         test_transaction.loc[index,col+'_rankcount'] = test_transaction.loc[index,col].map(col_count)

# train_transaction["addr2"] = train_transaction["addr1"].map(str) + "_" + train_transaction["addr2"].map(str)
# test_transaction["addr2"] = test_transaction["addr1"].map(str) + "_" + test_transaction["addr2"].map(str)
    
#     col_count = train_test.groupby(col)['TransactionDT'].count().fillna(0)
#     col_count = col_count.reset_index()
#     col_count.columns = col + ["_".join(col)+'_count']
#     index = train_transaction.index
#     train_transaction = train_transaction.merge(col_count, how='left', on = col)
#     train_transaction.index = index
#     index = test_transaction.index
#     test_transaction = test_transaction.merge(col_count, how='left', on = col)
#     test_transaction.index = index


for col in "card1,card2,card3,card4,card5,addr1".split(","):
    col_count = train_test.groupby(col)['TransactionAmt'].mean()
    train_transaction[col+'_amtcount'] = train_transaction[col].map(col_count)
    test_transaction[col+'_amtcount'] = test_transaction[col].map(col_count)
    col_count = train_test.groupby(col)['TransactionAmt'].std()
    train_transaction[col+'_amtscount'] = train_transaction[col].map(col_count)
    test_transaction[col+'_amtscount'] = test_transaction[col].map(col_count)
    
    for col2 in "C5,C8".split(','):
        col_count1 = train_test[train_test[col2] == 0].groupby(col)[col2].count()
        col_count2 = train_test[train_test[col2] != 0].groupby(col)[col2].count()
        train_transaction[col+'_'+col2+'count'] = train_transaction[col].map(col_count2) / (train_transaction[col].map(col_count1) + 0.01)
        test_transaction[col+'_'+col2+'count'] = test_transaction[col].map(col_count2) / (test_transaction[col].map(col_count1) + 0.01)
    for col2 in "C13".split(','):
        col_count1 = train_test[train_test[col2] == 1].groupby(col)[col2].count()
        col_count2 = train_test[train_test[col2] != 1].groupby(col)[col2].count()
        train_transaction[col+'_'+col2+'count'] = train_transaction[col].map(col_count2) / (train_transaction[col].map(col_count1) + 0.01)
        test_transaction[col+'_'+col2+'count'] = test_transaction[col].map(col_count2) / (test_transaction[col].map(col_count1) + 0.01)
        
        #     col_count1 = train_test[train_test['C14'] == 0].groupby(col)['C14'].count()
#     col_count2 = train_test[train_test['C14'] != 0].groupby(col)['C14'].count()
#     train_transaction[col+'_C14count'] = train_transaction[col].map(col_count2) / (train_transaction[col].map(col_count1) + 0.01)
#     test_transaction[col+'_C14count'] = test_transaction[col].map(col_count2) / (test_transaction[col].map(col_count1) + 0.01)
# for col in "card1,card2,card3,card5,card6,addr1,addr2,C1,C2".split(","):
#     col_count = train_test.groupby(col)['TransactionDT'].first()
#     train_transaction[col+'_first_dt'] = train_transaction[col].map(col_count) - train_transaction['TransactionDT']
#     test_transaction[col+'_first_dt'] = test_transaction[col].map(col_count) - test_transaction['TransactionDT']
#     col_count = train_test.groupby(col)['TransactionDT'].last()
#     train_transaction[col+'_last_dt'] = train_transaction[col].map(col_count) - train_transaction['TransactionDT']
#     test_transaction[col+'_last_dt'] = test_transaction[col].map(col_count) - test_transaction['TransactionDT']
    
#     print(col,len(set(train_transaction[col].tolist())),len(set(test_transaction[col].tolist())),len(set(test_transaction[col].tolist()) - set(train_transaction[col].tolist())))

train_transaction['TransactionAmt_to_mean_card1'] = train_transaction['TransactionAmt'] / train_transaction.groupby(['card1'])['TransactionAmt'].transform('mean')
train_transaction['TransactionAmt_to_mean_card4'] = train_transaction['TransactionAmt'] / train_transaction.groupby(['card4'])['TransactionAmt'].transform('mean')
train_transaction['TransactionAmt_to_std_card1'] = train_transaction['TransactionAmt'] / train_transaction.groupby(['card1'])['TransactionAmt'].transform('std')
train_transaction['TransactionAmt_to_std_card4'] = train_transaction['TransactionAmt'] / train_transaction.groupby(['card4'])['TransactionAmt'].transform('std')
test_transaction['TransactionAmt_to_mean_card1'] = test_transaction['TransactionAmt'] / test_transaction.groupby(['card1'])['TransactionAmt'].transform('mean')
test_transaction['TransactionAmt_to_mean_card4'] = test_transaction['TransactionAmt'] / test_transaction.groupby(['card4'])['TransactionAmt'].transform('mean')
test_transaction['TransactionAmt_to_std_card1'] = test_transaction['TransactionAmt'] / test_transaction.groupby(['card1'])['TransactionAmt'].transform('std')
test_transaction['TransactionAmt_to_std_card4'] = test_transaction['TransactionAmt'] / test_transaction.groupby(['card4'])['TransactionAmt'].transform('std')

train_transaction['D15_to_mean_card1'] = train_transaction['D15'] / train_transaction.groupby(['card1'])['D15'].transform('mean')
train_transaction['D15_to_mean_card4'] = train_transaction['D15'] / train_transaction.groupby(['card4'])['D15'].transform('mean')
train_transaction['D15_to_std_card1'] = train_transaction['D15'] / train_transaction.groupby(['card1'])['D15'].transform('std')
train_transaction['D15_to_std_card4'] = train_transaction['D15'] / train_transaction.groupby(['card4'])['D15'].transform('std')

test_transaction['D15_to_mean_card1'] = test_transaction['D15'] / test_transaction.groupby(['card1'])['D15'].transform('mean')
test_transaction['D15_to_mean_card4'] = test_transaction['D15'] / test_transaction.groupby(['card4'])['D15'].transform('mean')
test_transaction['D15_to_std_card1'] = test_transaction['D15'] / test_transaction.groupby(['card1'])['D15'].transform('std')
test_transaction['D15_to_std_card4'] = test_transaction['D15'] / test_transaction.groupby(['card4'])['D15'].transform('std')

train_transaction['D15_to_mean_addr1'] = train_transaction['D15'] / train_transaction.groupby(['addr1'])['D15'].transform('mean')
train_transaction['D15_to_std_addr1'] = train_transaction['D15'] / train_transaction.groupby(['addr1'])['D15'].transform('std')

test_transaction['D15_to_mean_addr1'] = test_transaction['D15'] / test_transaction.groupby(['addr1'])['D15'].transform('mean')
test_transaction['D15_to_std_addr1'] = test_transaction['D15'] / test_transaction.groupby(['addr1'])['D15'].transform('std')
# card1 13553 13244 3538
# card2 9433 9151 8655
# card3 1679 3117 3021
# card4 5 5 0
# card5 4378 4649 4566
# card6 5 4 0
# addr1 66038 65901 65718
# addr2 65780 65681 65628
# C1 1657 1177 533
# C2 1216 1126 571
# C3 27 34 8
# C4 1260 731 275
# C5 319 356 37
# C6 1328 837 249
# C7 1103 521 235
# C8 1253 445 169
# C9 205 357 158
# C10 1231 459 159
# C11 1476 940 361
# C12 1199 666 334
# C13 1597 5817 4842
# C14 1108 572 45


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [46]:
train_transaction['uid'] = train_transaction['card1'].astype(str)+'_'+train_transaction['card2'].astype(str)+'_'+train_transaction['card3'].astype(str)+'_'+train_transaction['card4'].astype(str)
test_transaction['uid'] = test_transaction['card1'].astype(str)+'_'+test_transaction['card2'].astype(str)+'_'+test_transaction['card3'].astype(str)+'_'+test_transaction['card4'].astype(str)

train_transaction['uid2'] = train_transaction['uid'].astype(str)+'_'+train_transaction['addr1'].astype(str)+'_'+train_transaction['addr2'].astype(str)
test_transaction['uid2'] = test_transaction['uid'].astype(str)+'_'+test_transaction['addr1'].astype(str)+'_'+test_transaction['addr2'].astype(str)

train_transaction['temp'] = train_transaction['TransactionDT2'].astype(str) + train_transaction['uid2'].astype(str) + train_transaction['TransactionAmt'].astype(str)
test_transaction['temp'] = test_transaction['TransactionDT2'].astype(str) + test_transaction['uid2'].astype(str) + test_transaction['TransactionAmt'].astype(str)



train_transaction['TransactionDT3'] = train_transaction['TransactionDT'].map(lambda x:(x//(3600*24*2)))
test_transaction['TransactionDT3'] = test_transaction['TransactionDT'].map(lambda x:(x//(3600*24*2)))

train_transaction['D16'] = train_transaction['D15'] - train_transaction['TransactionDT'].map(lambda x:(x//(3600*24)))
test_transaction['D16'] = test_transaction['D15'] - test_transaction['TransactionDT'].map(lambda x:(x//(3600*24)))

for col in "D1,D2,D3,D4,D10".split(','):
    train_transaction[col + "_day"] = train_transaction[col] - train_transaction['TransactionDT'].map(lambda x:(x//(3600*24)))
    test_transaction[col + "_day"] = test_transaction[col] - test_transaction['TransactionDT'].map(lambda x:(x//(3600*24)))
    

col_del_amt = []
for col in [['TransactionDT2','uid2','TransactionAmt']]: 

    train_transaction['TransactionAmt_daysum'] = train_transaction.groupby('temp')['TransactionAmt'].transform('sum')
    test_transaction['TransactionAmt_daysum'] = test_transaction.groupby('temp')['TransactionAmt'].transform('sum')
    col_del_amt.append('TransactionAmt_daysum')
#     col_count = train_transaction.groupby('temp')['ukey'].apply(entropy)
#     train_transaction['TransactionAmt_ukeyen'] = train_transaction['temp'].map(col_count)
#     col_count = test_transaction.groupby('temp')['ukey'].apply(entropy)
#     test_transaction['TransactionAmt_ukeyen'] = test_transaction['temp'].map(col_count)
    col_del_amt.append('TransactionAmt_ukeyen')
    for col2 in "C8,C13,D15".split(','):
        col_count1 = train_transaction[train_transaction[col2] == 0].groupby('temp')[col2].count()
        train_transaction["_".join(col)+'_'+col2+'count'] = (train_transaction['temp'].map(col_count1) /
                                                             train_transaction.fillna(-1).groupby('temp')['TransactionAmt'].transform('count'))
        col_count1 = test_transaction[test_transaction[col2] == 0].groupby('temp')[col2].count()
        test_transaction["_".join(col)+'_'+col2+'count'] = (test_transaction['temp'].map(col_count1)/
                                                             test_transaction.fillna(-1).groupby('temp')['TransactionAmt'].transform('count'))
        col_del_amt.append("_".join(col)+'_'+col2+'count')
        #         print(train_transaction["_".join(col)+'_'+col2+'count'])

    for col2 in "D1_day,D4_day,D15,D16".split(','):
        train_transaction["_".join(col)+'_'+col2+'mcount'] = (
                                                             train_transaction.groupby('temp')[col2].transform('mean'))
        test_transaction["_".join(col)+'_'+col2+'mcount'] = (
                                                             test_transaction.groupby('temp')[col2].transform('mean'))
        col_del_amt.append("_".join(col)+'_'+col2+'mcount')
    for col2 in "C1,C5,C8,D1,D4_day,D15,D16,V307,V310,V127".split(','):
        train_transaction["_".join(col)+'_'+col2+'scount'] = (
                                                             train_transaction.groupby('temp')[col2].transform('std'))
        test_transaction["_".join(col)+'_'+col2+'scount'] = (
                                                             test_transaction.groupby('temp')[col2].transform('std'))
        col_del_amt.append("_".join(col)+'_'+col2+'scount')
        
col_del_ukey2 = []
for col in [['tempkey']]: 
    train_transaction['TransactionAmt_ukeycount'] = train_transaction.groupby('ukey')['TransactionAmt'].transform('count')
    test_transaction['TransactionAmt_ukeycount'] = test_transaction.groupby('ukey')['TransactionAmt'].transform('count')
    col_del_ukey2.append('TransactionAmt_ukeysum')
    
#     for col2 in "addr1".split(','):    
#         col_count1 = train_transaction[train_transaction[col2].isna()].groupby('temp')[col2].count()
#         train_transaction["_".join(col)+'_'+col2+'count'] = (train_transaction['temp'].map(col_count1) /
#                                                              train_transaction.fillna(-1).groupby(col)['TransactionAmt'].transform('count'))
#         col_count1 = test_transaction[test_transaction[col2].isna()].groupby('temp')[col2].count()
#         test_transaction["_".join(col)+'_'+col2+'count'] = (test_transaction['temp'].map(col_count1) /
#                                                              test_transaction.fillna(-1).groupby(col)['TransactionAmt'].transform('count'))
#         print(train_transaction["_".join(col)+'_'+col2+'count'])
#         print(train_transaction['en_' + "_".join(col)+'_'+col2+ '_count'])

train_transaction['TransactionDT3'] = train_transaction['TransactionDT'].map(lambda x:(x//(3600*2)))
test_transaction['TransactionDT3'] = test_transaction['TransactionDT'].map(lambda x:(x//(3600*2)))
col_del_ukey = []
for col in [['ukey']]: 
    train_transaction['TransactionAmt_ukeysum'] = train_transaction.groupby('ukey')['TransactionAmt'].transform('sum')
    test_transaction['TransactionAmt_ukeysum'] = test_transaction.groupby('ukey')['TransactionAmt'].transform('sum')
    train_transaction['TransactionAmt_ukeystd'] = train_transaction.groupby('ukey')['TransactionAmt'].transform('std')
    test_transaction['TransactionAmt_ukeystd'] = test_transaction.groupby('ukey')['TransactionAmt'].transform('std')
#     train_transaction['TransactionAmt_ukeycount'] = train_transaction.groupby('ukey')['TransactionAmt'].transform('count')
#     test_transaction['TransactionAmt_ukeycount'] = test_transaction.groupby('ukey')['TransactionAmt'].transform('count')
    col_del_ukey.append('TransactionAmt_ukeysum')
    col_del_ukey.append('TransactionAmt_ukeystd')
    
    for col2 in "C3,C5,C7,C13,D1,D15".split(','):
        col_count1 = train_transaction[train_transaction[col2] == 0].groupby('ukey')[col2].count()
        train_transaction["_".join(col)+'_'+col2+'count'] = (train_transaction['ukey'].map(col_count1) /
                                                             train_transaction.groupby(col)['TransactionAmt'].transform('count'))
        col_count1 = test_transaction[test_transaction[col2] == 0].groupby('ukey')[col2].count()
        test_transaction["_".join(col)+'_'+col2+'count'] = (test_transaction['ukey'].map(col_count1)/
                                                             test_transaction.groupby(col)['TransactionAmt'].transform('count'))
        col_del_ukey.append("_".join(col)+'_'+col2+'count')
        
    for col2 in "C1,C4,C13,D10_day,D4_day,D16".split(','):
        train_transaction["_".join(col)+'_'+col2+'scount'] = (
                                                             train_transaction.groupby(col)[col2].transform('std'))
        test_transaction["_".join(col)+'_'+col2+'scount'] = (
                                                             test_transaction.groupby(col)[col2].transform('std'))
        col_del_ukey.append("_".join(col)+'_'+col2+'scount')
        

    


del train_transaction['ukey'],test_transaction['ukey']

train_test = train_transaction.append(test_transaction)
for i in ['D1_day','D2_day','D4_day']:
    train_test[str(i)+'C13_MEAN']  = train_test[i]/train_test.groupby(['uid2',i])['C13'].transform('mean')
    train_test[str(i)+'C13_STD']   = train_test[i]/train_test.groupby(['uid2',i])['C13'].transform('std')
    train_test[str(i)+'C13_sum']   = train_test.groupby(['uid2',i])['C13'].transform('sum')
    train_test[str(i)+'C13_max']   = train_test.groupby(['uid2',i])['C13'].transform('max')
    train_test[str(i)+'C13_min']   = train_test.groupby(['uid2',i])['C13'].transform('min')
    train_test[str(i)+'C13_cnt']   = train_test.groupby(['uid2',i])['C13'].transform('count')
    
    train_transaction[str(i)+'C13_MEAN'] = train_test.loc[train_transaction.index,str(i)+'C13_MEAN']
    train_transaction[str(i)+'C13_STD'] = train_test.loc[train_transaction.index,str(i)+'C13_STD']
    train_transaction[str(i)+'C13_sum'] = train_test.loc[train_transaction.index,str(i)+'C13_sum']
    train_transaction[str(i)+'C13_max'] = train_test.loc[train_transaction.index,str(i)+'C13_max']
    train_transaction[str(i)+'C13_min'] = train_test.loc[train_transaction.index,str(i)+'C13_min']
    train_transaction[str(i)+'C13_cnt'] = train_test.loc[train_transaction.index,str(i)+'C13_cnt']
    test_transaction[str(i)+'C13_MEAN'] = train_test.loc[test_transaction.index,str(i)+'C13_MEAN']
    test_transaction[str(i)+'C13_STD'] = train_test.loc[test_transaction.index,str(i)+'C13_STD']
    test_transaction[str(i)+'C13_sum'] = train_test.loc[test_transaction.index,str(i)+'C13_sum']
    test_transaction[str(i)+'C13_max'] = train_test.loc[test_transaction.index,str(i)+'C13_max']
    test_transaction[str(i)+'C13_min'] = train_test.loc[test_transaction.index,str(i)+'C13_min']
    test_transaction[str(i)+'C13_cnt'] = train_test.loc[test_transaction.index,str(i)+'C13_cnt']
    
    
for i in ['D1_day','D2_day','D4_day']:
    train_test[str(i)+'D_sum']  = train_test.groupby(['uid2',i])['TransactionAmt'].transform('sum')
    train_test[str(i)+'D_max']  = train_test.groupby(['uid2',i])['TransactionAmt'].transform('max')
    train_test[str(i)+'D_min']  = train_test.groupby(['uid2',i])['TransactionAmt'].transform('min')
    train_test[str(i)+'D_cnt']  = train_test.groupby(['uid2',i])['TransactionAmt'].transform('count')
    
    train_transaction[str(i)+'D_sum'] = train_test.loc[train_transaction.index,str(i)+'D_sum']
    train_transaction[str(i)+'D_max'] = train_test.loc[train_transaction.index,str(i)+'D_max']
    train_transaction[str(i)+'D_min'] = train_test.loc[train_transaction.index,str(i)+'D_min']
    train_transaction[str(i)+'D_cnt'] = train_test.loc[train_transaction.index,str(i)+'D_cnt']
    test_transaction[str(i)+'D_sum'] = train_test.loc[test_transaction.index,str(i)+'D_sum']
    test_transaction[str(i)+'D_max'] = train_test.loc[test_transaction.index,str(i)+'D_max']
    test_transaction[str(i)+'D_min'] = train_test.loc[test_transaction.index,str(i)+'D_min']
    test_transaction[str(i)+'D_cnt'] = train_test.loc[test_transaction.index,str(i)+'D_cnt']

del train_transaction['tempkey'],test_transaction['tempkey']
del train_transaction['temp'],test_transaction['temp']
del train_transaction['uid'],test_transaction['uid']
del train_transaction['uid2'],test_transaction['uid2']
del train_transaction['TransactionDT3'],test_transaction['TransactionDT3']

In [47]:
for col in "C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14".split(","):
#     print(col,train_test[col].value_counts())
    col_count = train_test.fillna(-1).groupby(col)['TransactionDT'].count()
    print(col_count)
    minnum = col_count[col_count == 1].index.min()
    print((col_count <= 2).sum(),col_count[col_count <= 2].index.max(),col_count[col_count <= 2].index.min(),col_count[col_count == 1].index.min())
#     train_transaction[col] = train_transaction[col].fillna(-1).map(lambda x:10000 if col_count[x] <= 2 else x)
#     test_transaction[col] = test_transaction[col].fillna(-1).map(lambda x:10000 if col_count[x] <= 2 else x)
    train_transaction[col] = train_transaction[col].fillna(-1).map(lambda x:minnum if x >= minnum else x)
    test_transaction[col] = test_transaction[col].fillna(-1).map(lambda x:minnum if x >= minnum  else x)

C1
-1.0            3
 0.0         5291
 1.0       586742
 2.0       197520
 3.0        96569
 4.0        53794
 5.0        32851
 6.0        19774
 7.0        12984
 8.0         9068
 9.0         6383
 10.0        4873
 11.0        3736
 12.0        3187
 13.0        2399
 14.0        1921
 15.0        1515
 16.0        1356
 17.0        1293
 18.0        1120
 19.0         894
 20.0         868
 21.0         712
 22.0         717
 23.0         492
 24.0         563
 25.0         525
 26.0         401
 27.0         431
 28.0         415
            ...  
 4629.0         1
 4630.0         1
 4631.0         1
 4632.0         1
 4633.0         1
 4635.0         1
 4642.0         1
 4643.0         1
 4644.0         1
 4645.0         3
 4651.0         1
 4653.0         1
 4654.0         1
 4655.0         1
 4658.0         2
 4660.0         4
 4661.0         2
 4662.0         1
 4663.0         3
 4665.0         1
 4666.0         1
 4667.0         2
 4668.0         1
 4669.0         1
 4670.0

C9
-1.0           3
 0.0      341561
 1.0      420354
 2.0      146643
 3.0       64015
 4.0       31672
 5.0       16483
 6.0       10670
 7.0        6499
 8.0        4712
 9.0        3351
 10.0       2128
 11.0       1871
 12.0       1395
 13.0       1123
 14.0        938
 15.0        786
 16.0        597
 17.0        522
 18.0        460
 19.0        524
 20.0        366
 21.0        369
 22.0        340
 23.0        251
 24.0        269
 25.0        200
 26.0        205
 27.0        182
 28.0        230
           ...  
 528.0         4
 535.0         2
 537.0         1
 539.0         1
 540.0         7
 543.0         2
 544.0         2
 545.0         1
 549.0         1
 550.0         2
 551.0         1
 553.0         1
 554.0         4
 555.0         2
 557.0         1
 558.0        10
 559.0         3
 560.0         2
 561.0         1
 562.0         1
 563.0         1
 564.0         6
 565.0         3
 566.0         9
 567.0         9
 568.0         1
 569.0         1
 570.0     

In [48]:
# train_test['card1'] = train_test['card1'].fillna(-999)
# colcount = train_test.groupby('card1')['TransactionAmt'].mean().sort_values()
# colcount[:] = np.array(range(colcount.shape[0]))
# train_transaction['card7'] = train_transaction['card1'].fillna(-999).map(colcount)
# test_transaction['card7'] = test_transaction['card1'].fillna(-999).map(colcount)

In [49]:
from sklearn.linear_model import LinearRegression 
print(train_test.columns)
col_del3 = []
for col in "D1,D2,D3,D4,D5,D6,D7,D8,D10,D11,D12,D13,D14,D15".split(","):
    df = train_test[['TransactionDT',col]]
    df = df[~df[col].isna()][df[col]>50.0]
    x = np.asarray(df[['TransactionDT']])*0.0001
    y = np.asarray(df[[col]])
    reg = LinearRegression().fit(x, y)
    print(col," Y = %.5fX + (%.5f)" % (reg.coef_[0][0], reg.intercept_[0]),x[0]*reg.coef_[0][0] + reg.intercept_[0],x[0],y[0])
    train_transaction[col+'_fix'] = train_transaction[col].fillna(-1)*reg.intercept_[0]/(train_transaction['TransactionDT'].map(lambda x:x *0.0001 * reg.coef_[0][0]) + reg.intercept_[0])
    train_transaction[col+'_fix'] = train_transaction[col+'_fix'].map(lambda x:-1 if x<0 else x)
    test_transaction[col+'_fix'] = test_transaction[col].fillna(-1)*reg.intercept_[0]/(test_transaction['TransactionDT'].map(lambda x:x *0.0001 * reg.coef_[0][0]) + reg.intercept_[0])
    test_transaction[col+'_fix'] = test_transaction[col+'_fix'].map(lambda x:-1 if x<0 else x)
    col_del3.append(col+'_fix')
#     train_test2 = train_transaction.append(test_transaction)
#     df = train_test2[['TransactionDT',col+'_fix']]
#     df = df[df[col+'_fix']>50.0]
#     x = np.asarray(df[['TransactionDT']])*0.0001
#     y = np.asarray(df[[col+'_fix']])
#     reg = LinearRegression().fit(x, y)
#     print(col," Y = %.5fX + (%.5f)" % (reg.coef_[0][0], reg.intercept_[0]),x[0]*reg.coef_[0][0] + reg.intercept_[0],x[0],y[0])
    
# for col in "D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15".split(","):
#     bin = 5
#     train_transaction[col+'_rankcount'] = 0
#     test_transaction[col+'_rankcount'] = 0
#     train_transaction[col] = train_transaction[col].fillna(-999).map(lambda x:int(x//2))
#     test_transaction[col] = test_transaction[col].fillna(-999).map(lambda x:int(x//2))
#     for i in range(bin):
        
#         index = train_transaction.iloc[train_transaction.shape[0] * i//5:(train_transaction.shape[0] * (i+1))//5,:].index
#         col_count = train_transaction.loc[index,:].groupby(col)['TransactionDT'].count().sort_values()
#         col_count[:] = np.array(range(col_count.shape[0]))/col_count.shape[0]
#         train_transaction.loc[index,col+'_rankcount'] = train_transaction.loc[index,col].map(col_count)
# #         print(train_transaction.loc[index,col+'_rankcount'])
#     for i in range(bin):
#         index = test_transaction.iloc[test_transaction.shape[0] * i//5:(test_transaction.shape[0] * (i+1))//5,:].index
#         col_count = test_transaction.loc[index,:].groupby(col)['TransactionDT'].count().sort_values()
#         col_count[:] = np.array(range(col_count.shape[0]))/col_count.shape[0]
#         test_transaction.loc[index,col+'_rankcount'] = test_transaction.loc[index,col].map(col_count)

Index(['C1', 'C10', 'C10_count', 'C11', 'C11_count', 'C12', 'C12_count', 'C13',
       'C13_count', 'C14',
       ...
       'D1_dayD_min', 'D1_dayD_cnt', 'D2_dayD_sum', 'D2_dayD_max',
       'D2_dayD_min', 'D2_dayD_cnt', 'D4_dayD_sum', 'D4_dayD_max',
       'D4_dayD_min', 'D4_dayD_cnt'],
      dtype='object', length=630)
D1  Y = 0.01384X + (248.77093) [248.89061746] [8.6499] [112.]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


D2  Y = 0.01344X + (246.70667) [246.82293357] [8.6499] [112.]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


D3  Y = 0.01731X + (132.12557) [132.27617065] [8.6994] [56.]


...

D13  Y = 0.01781X + (185.28074) [185.44756472] [9.3654] [58.]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


D14  Y = 0.04181X + (267.52917) [267.89804867] [8.8229] [98.]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


D15  Y = 0.04879X + (282.36472) [282.7865558] [8.6469] [315.]


In [50]:
train_test = train_identity.append(test_identity).fillna(-1)    
for col in "id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo".split(","):
    navalue = -1
    if col in ['id_01','id_03','id_04','id_05','id_06','id_07','id_08','id_09','id_10']:
        navalue = -999
    col_count = train_test.fillna(navalue).groupby(col)['id_01'].count()
    train_identity[col+'_count'] = train_identity[col].fillna(navalue).map(col_count)
    test_identity[col+'_count'] = test_identity[col].fillna(navalue).map(col_count)
    print(col_count.sort_values())

id_01
-89.0          1
-73.0          1
-82.0          1
-83.0          1
-84.0          1
-69.0          1
-86.0          1
-78.0          1
-77.0          1
-91.0          1
-41.0          1
-94.0          1
-36.0          1
-24.0          1
-49.0          1
-52.0          2
-53.0          2
-43.0          2
-39.0          2
-48.0          2
-54.0          2
-93.0          2
-99.0          2
-92.0          3
-57.0          3
-72.0          3
-58.0          3
-66.0          3
-76.0          3
-51.0          3
           ...  
-7.0          18
-17.0         20
-21.0         23
-12.0         24
-6.0          26
-26.0         30
-18.0         32
-16.0         38
-11.0         39
-75.0        196
-70.0        205
-85.0        249
-90.0        323
-65.0        361
-80.0        373
-55.0        576
-95.0        698
-60.0       1596
-50.0       1600
-100.0      1706
-30.0       1754
-40.0       2409
-35.0       2809
-45.0       6695
-25.0       9640
-15.0      12722
-20.0      22341
-10.0   

id_12
Found        42220
NotFound    243920
Name: id_01, dtype: int64
id_13
 34.0         1
 50.0         1
 40.0         1
 29.0         1
 23.0         1
 59.0         1
 10.0         1
 12.0         2
 26.0         2
 46.0         3
 47.0         3
 60.0         3
 37.0         3
 57.0         3
 17.0         3
 16.0         3
 53.0         4
 22.0         5
 21.0         8
 30.0         9
 42.0         9
 32.0        10
 48.0        12
 58.0        17
 56.0        19
 13.0        21
 38.0        22
 36.0        25
 45.0        26
 31.0        38
 39.0        45
 54.0        67
 28.0        77
 44.0        87
 61.0       108
 35.0       317
 51.0       349
 15.0       520
 24.0       531
 55.0       781
 11.0       890
 43.0       907
 41.0      1086
 19.0      1147
 18.0      1218
 25.0      1302
 62.0      1752
 63.0      2050
 20.0      3804
 14.0      6427
 33.0     10048
 64.0     14429
 49.0     26365
-1.0      28534
 27.0     73282
 52.0    109760
Name: id_01, dtype: int64
id

id_26
 210.0         1
 127.0         1
 172.0         1
 173.0         1
 132.0         1
 175.0         1
 181.0         1
 187.0         1
 115.0         1
 207.0         1
 112.0         1
 160.0         1
 154.0         1
 214.0         1
 196.0         1
 101.0         1
 105.0         1
 139.0         1
 104.0         1
 198.0         1
 193.0         2
 170.0         2
 125.0         2
 123.0         2
 140.0         2
 179.0         2
 145.0         2
 195.0         2
 165.0         2
 208.0         3
           ...  
 177.0        36
 134.0        39
 138.0        40
 156.0        42
 106.0        44
 163.0        45
 150.0        48
 133.0        53
 143.0        56
 149.0        67
 168.0        74
 117.0        78
 162.0        89
 194.0       120
 190.0       135
 121.0       176
 146.0       179
 182.0       225
 215.0       228
 216.0       352
 169.0       410
 137.0       441
 147.0       547
 102.0       583
 119.0       669
 100.0       891
 142.0      1126
 184.0  

In [51]:
train = train_transaction.merge(train_identity.fillna(-1) , how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity.fillna(-1) , how='left', left_index=True, right_index=True)
train_test = train.append(test).fillna(-1)
for col in "card1,card2,card5".split(","):
    
    for col2 in "id_01".split(','):
        col_count1 = train_test[train_test[col2].isna()].groupby(col)[col].count()
        col_count2 = train_test.groupby(col)[col].count()
        train_transaction[col+'_'+col2+'count'] = train_transaction[col].map(col_count1) / (train_transaction[col].map(col_count2) + 0.01)
        test_transaction[col+'_'+col2+'count'] = test_transaction[col].map(col_count1) / (test_transaction[col].map(col_count2) + 0.01)

# train_identity['id_02_to_mean_card1'] = train_identity['id_02'] / train.groupby(['card1'])['id_02'].transform('mean')
# test_identity['id_02_to_mean_card1'] = test_identity['id_02'] / test.groupby(['card1'])['id_02'].transform('mean')

# train_identity['id_02_mean_card1'] =train.groupby(['card1'])['id_02'].transform('mean')
# test_identity['id_02_mean_card1'] = test.groupby(['card1'])['id_02'].transform('mean')
del train_test,train,test

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [52]:
col_del = []
for i in range(339):
    col = "V" + str(i+1)
    s = train_transaction[col].fillna(0).map(lambda x:0 if x%1 == 0 else 1).sum()
    if s > 100:
        print(col,s)
        col_del.append(col)
#         if (i + 1<263 and i + 1 >= 215) or i + 1 < 159 or i + 1 >= 276:
#             train_transaction[col+"_fix"] = train_transaction[col]/train_transaction['TransactionAmt']
#             test_transaction[col+"_fix"] = test_transaction[col]/test_transaction['TransactionAmt']
#     del train_transaction[col],test_transaction[col]

print(train_transaction[train_transaction['C5'] == 0].isFraud.value_counts())
print(train_transaction[train_transaction['C5'] == 1].isFraud.value_counts())
print(train_transaction[train_transaction['C14'] == 0].isFraud.value_counts())
print(train_transaction[train_transaction['C14'] == 1].isFraud.value_counts())
print(train_transaction.V70.fillna(7).value_counts())
print(train_transaction[train_transaction['V70'] == 0].isFraud.value_counts())
print(train_transaction[train_transaction['V70'].isna()].isFraud.value_counts())

V126 24875
V127 104743
V128 46447
V129 8376
V130 89202
V131 29537
V132 9116
V133 14910
V134 11134
V135 9184
V136 14824
V137 10682
V159 8116
V160 10836
V164 597
V165 3440
V166 2347
V202 16120
V203 20278
V204 18198
V205 6576
V206 4897
V207 9195
V208 10054
V209 10466
V210 10210
V211 10131
V212 11253
V213 10710
V214 4277
V215 5191
V216 4769
V263 16586
V264 20089
V265 18161
V266 6886
V267 10591
V268 8642
V270 8138
V271 8432
V272 8261
V273 10568
V274 11959
V275 11161
V276 4398
V277 5417
V278 4856
V306 37081
V307 140538
V308 67074
V309 16636
V310 121258
V311 10140
V312 46506
V313 50088
V314 59747
V315 51521
V316 15081
V317 24841
V318 18718
V319 10762
V320 18664
V321 13113
0    353676
1     18759
Name: isFraud, dtype: int64
0    121954
1       929
Name: isFraud, dtype: int64
0    30733
1     5214
Name: isFraud, dtype: int64
0    311589
1      8600
Name: isFraud, dtype: int64
0.0    317801
1.0    184177
7.0     77096
2.0      9739
3.0      1296
4.0       253
5.0       177
6.0         1
Name: V7

In [53]:
# train_identity['has_identity'] = 1
# test_identity['has_identity'] = 1
train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)
# train = pd.concat([train,f_prone_train],axis = 1)
# test = pd.concat([test,f_prone_test],axis = 1)
# train['has_identity'] = train['has_identity'].fillna(0)
# test['has_identity'] = test['has_identity'].fillna(0)
# train = train.merge(train_f5, how='left', left_index=True, right_index=True)
# test = test.merge(test_f5, how='left', left_index=True, right_index=True)
# train = train.merge(train_f6, how='left', left_index=True, right_index=True)
# test = test.merge(test_f6, how='left', left_index=True, right_index=True)
train[train['isFraud'] == 1].to_csv('./fraud.csv')
print(train.shape)
print(test.shape)

# train['TransactionDT2'] = train['TransactionDT'].map(lambda x:(x//(3600*24*7)))
# test['TransactionDT2'] = test['TransactionDT'].map(lambda x:(x//(3600*24*7)))


test['isFraud'] = 0
y_train = train['isFraud'].copy()
y_test = test['isFraud'].copy()
del train_identity, test_identity
del train_transaction, test_transaction
# del train['card1'],test['card1'],train['card2'],test['card2']
del train['TransactionDT'],test['TransactionDT']
del train['TransactionDT2'],test['TransactionDT2']

# Drop target, fill in NaNs
X_train = train.drop('isFraud', axis=1)
X_test = test.drop('isFraud', axis=1)

del test

# Label Encoding
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        print(f)
        if f in ['id_17','id_19','id_20','id_21','id_25','id_26']:
            X_train[f] = X_train[f].fillna(-1)
            X_test[f] = X_test[f].fillna(-1)
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))   

(590540, 721)
(506691, 720)
ProductCD
card4
card6
P_emaildomain
R_emaildomain
M1
M2
M3
M4
M5
M6
M7
M8
M9
id_31_raw
id_30_raw
DeviceInfo_raw
id_12
id_15
id_16
id_23
id_27
id_28
id_29
id_30
id_31
id_33
id_34
id_35
id_36
id_37
id_38
DeviceType
DeviceInfo


In [54]:
print(X_train.shape,X_test.shape)

(590540, 718) (506691, 718)


In [55]:
%%time
# From kernel https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
# WARNING! THIS CAN DAMAGE THE DATA 
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

debug = False

X_train = reduce_mem_usage(X_train)
if not debug:
    X_test = reduce_mem_usage(X_test)


if debug:
    split_pos = X_train.shape[0]*4//5
    y_test = y_train.iloc[split_pos:]
    y_train = y_train.iloc[:split_pos]
    X_test = X_train.iloc[split_pos:,:]
    X_train = X_train.iloc[:split_pos,:]
    


Memory usage of dataframe is 3259.43 MB
Memory usage after optimization is: 859.83 MB
Decreased by 69.3%
CPU times: user 7min 53s, sys: 19min 52s, total: 27min 46s
Wall time: 27min 46s


In [56]:
# # X_train2 = X_train.copy()
# # X_train2['isFraud'] = y_train
# # X_train2['hour2'] = X_train2['TransactionDT'].map(lambda x:(x//(3600*24*2)))
# # X_train2 = X_train2.drop_duplicates(subset = ['hour2','card1','TransactionAmt','isFraud'])
# # y_train2 = X_train2['isFraud']
# # del X_train2['isFraud'],X_train2['hour2'] 


# temp = X_train[['TransactionDT','TransactionAmt']].values
# folds_index = [([],[]),([],[]),([],[])]
# cache = {}
# for i in range(temp.shape[0]):
#     amt = temp[i,1]
#     cache[amt] = cache.get(amt,[])
#     cache[amt].append(i)
#     l = len(cache[amt])
#     folds_num = (int(amt*1000)+l)%3
#     folds_index[folds_num][1].append(i)
#     for j in range(3):
#         if j != folds_num:
#             folds_index[j][0].append(i)

# for i in range(3):
#     folds_index[i] = (np.array(folds_index[i][0]) ,np.array(folds_index[i][1]))
#     print(folds_index[i],len(folds_index[i][0]))
    


In [57]:
def metric(true,pred):
    thre = 0.2
    pred = (pred > thre).astype(int)
    p = np.sum(pred*true)/(np.sum(pred) + 0.01)
    r = np.sum(pred*true)/(np.sum(true) + 0.01)
    return p,r,2*p*r/(p+r)

In [58]:
%%time

from sklearn.model_selection import KFold,TimeSeriesSplit
from sklearn.metrics import roc_auc_score
folds = 3
    
kf = KFold(n_splits = folds, shuffle = True, random_state=seed)
features = [x for x in X_train.columns]

y_preds = np.zeros(X_test.shape[0])
y_oof = np.zeros(X_train.shape[0])
i = 0
params = {
            "n_estimators":700,
        "max_depth":15,
        "learning_rate":0.03,
        "subsample":0.9,
        "colsample_bytree":0.7,
        "min_child_weight":2.5,
        "tree_method":'gpu_hist',
                'reg_alpha' :0.5,
            'reg_lambda' : 0.4,
}
for tr_idx, val_idx in kf.split(X_train, y_train):
# for tr_idx, val_idx in folds_index:
    print(val_idx)
    i+=1
    clf = xgb.XGBClassifier(
        **params
    )
    
    X_tr, X_vl = X_train[features].iloc[tr_idx, :], X_train[features].iloc[val_idx, :]
    y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
    clf.fit(X_tr, y_tr)
    y_pred_train = clf.predict_proba(X_vl)[:,1]
    y_oof[val_idx] = y_pred_train
    print('ROC AUC {}'.format(roc_auc_score(y_vl, y_pred_train)))
    
    temp = clf.predict_proba(X_test[features])[:,1]
    y_preds+= temp / folds
    if debug:    
        print("debug:",roc_auc_score(y_test, temp),log_loss(y_test, temp),metric(y_test, temp))    
    del X_tr,X_vl
        
if debug:    
    print("debug:",roc_auc_score(y_test, y_preds),log_loss(y_test, y_preds),metric(y_test, y_preds))  




[     0      5      7 ... 590531 590538 590539]
ROC AUC 0.9822539159229746
[     1      3     12 ... 590534 590535 590537]
ROC AUC 0.9808598679058511
[     2      4      6 ... 590524 590526 590536]
ROC AUC 0.9802236868621235
CPU times: user 5min 54s, sys: 3min 7s, total: 9min 2s
Wall time: 17min 21s


In [59]:
features = [x for x in X_train.columns if (x not in col_del_ukey and x not in ['key4_std'] and not x.startswith("w2v") and x not in ['TransactionDT'] )]
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
folds = 3
kf = KFold(n_splits = folds, shuffle = True, random_state=seed)
y_preds11 = np.zeros(X_test.shape[0])
y_oof = np.zeros(X_train.shape[0])
i = 0
for tr_idx, val_idx in kf.split(X_train, y_train):
    i+=1
    clf = xgb.XGBClassifier(
        n_estimators=700,
        max_depth=9,
        learning_rate=0.03,
        subsample=0.9,
        colsample_bytree=0.9,
        tree_method='gpu_hist'
    )
    
    
    X_tr, X_vl = X_train[features].iloc[tr_idx, :], X_train[features].iloc[val_idx, :]
    y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
    clf.fit(X_tr, y_tr)
    y_pred_train = clf.predict_proba(X_vl)[:,1]
    y_oof[val_idx] = y_pred_train
    print('ROC AUC {}'.format(roc_auc_score(y_vl, y_pred_train)))
    
    temp = clf.predict_proba(X_test[features])[:,1] 
    y_preds11+= temp/ folds
    if debug:    
        print("debug:",roc_auc_score(y_test, temp),log_loss(y_test, temp),metric(y_test, temp))   
    del X_tr,X_vl
        

if debug:    
    print("debug:",roc_auc_score(y_test, y_preds11),log_loss(y_test, y_preds11),metric(y_test, y_preds11))  
    print("debug:",roc_auc_score(y_test, y_preds11*0.5 + y_preds*0.5),log_loss(y_test, y_preds11*0.5 + y_preds*0.5),metric(y_test, y_preds11*0.5 + y_preds*0.5))


ROC AUC 0.9765437149403929
ROC AUC 0.9755468884201014
ROC AUC 0.9755194058309661


In [60]:
%%time

from sklearn.model_selection import KFold,TimeSeriesSplit
from sklearn.metrics import roc_auc_score
folds = 3
    
kf = KFold(n_splits = folds, shuffle = False, random_state=seed)
features = [x for x in X_train.columns]

y_preds = np.zeros(X_test.shape[0])
y_oof = np.zeros(X_train.shape[0])
X_test_pred = pd.DataFrame(index = X_test.index)
X_train_pred = pd.DataFrame(index = X_train.index)
X_test_pred['pred_3'] = 0
X_test_pred['pred_1'] = 0
X_test_pred['pred_2'] = 0
i = 0
params = {
            "n_estimators":700,
        "max_depth":15,
        "learning_rate":0.03,
        "subsample":0.9,
        "colsample_bytree":0.7,
        "min_child_weight":2.5,
        "tree_method":'gpu_hist',
                'reg_alpha' :0.5,
            'reg_lambda' : 0.4,
}
for tr_idx, val_idx in kf.split(X_train, y_train):
# for tr_idx, val_idx in folds_index:
    print(val_idx)
    i+=1
    clf = xgb.XGBClassifier(
        **params
    )
    
    X_tr, X_vl = X_train[features].iloc[tr_idx, :], X_train[features].iloc[val_idx, :]
    y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
    clf.fit(X_tr, y_tr)
    y_pred_train = clf.predict_proba(X_vl)[:,1]
    y_oof[val_idx] = y_pred_train
    print('ROC AUC {}'.format(roc_auc_score(y_vl, y_pred_train)))
    
    temp = clf.predict_proba(X_test[features])[:,1]
    y_preds+= temp / folds
    X_test_pred['pred_1'] += temp / folds
    if debug:    
        print("debug:",roc_auc_score(y_test, temp),log_loss(y_test, temp),metric(y_test, temp))    
    del X_tr,X_vl
        
if debug:    
    print("debug:",roc_auc_score(y_test, y_preds),log_loss(y_test, y_preds),metric(y_test, y_preds))  


X_train_pred['pred_1'] = y_oof


[     0      1      2 ... 196844 196845 196846]
ROC AUC 0.9251000785048729
[196847 196848 196849 ... 393691 393692 393693]
ROC AUC 0.9446345265731108
[393694 393695 393696 ... 590537 590538 590539]
ROC AUC 0.9346126962478157
CPU times: user 5min 36s, sys: 3min, total: 8min 37s
Wall time: 17min 3s


In [61]:
temp = X_train.loc[2987367:2987368]
temp2 = X_train.loc[2987366:2987367]
temp11 = train.loc[2987367:2987368]
temp22 = train.loc[2987366:2987367]
temp3 = temp.copy()
# print(temp.values)
# clf.predict_proba(X_train.loc[2990196:2990197])[:,1]
print(X_train.V70.value_counts())
for i in range(X_train.shape[1]-1,-1,-1):
    
    temp3.iloc[0,i] = temp2.iloc[0,i]
    print(i,X_train.columns[i],temp.iloc[0,i],temp2.iloc[0,i],temp11[X_train.columns[i]].iloc[0],temp22[X_train.columns[i]].iloc[0],clf.predict_proba(temp3)[:,1])
    
# C5 C14 V70

0.0    317801
1.0    184177
2.0      9739
3.0      1296
4.0       253
5.0       177
6.0         1
Name: V70, dtype: int64
717 DeviceInfo_count 378.0 38502.0 378.0 38502.0 [0.9934515  0.00464595]
716 DeviceType_count 159568.0 118173.0 159568.0 118173.0 [0.99354446 0.00464595]
715 id_38_count 168980.0 108982.0 168980.0 108982.0 [0.9933448  0.00464595]
714 id_37_count 215149.0 215149.0 215149.0 215149.0 [0.9933448  0.00464595]
713 id_36_count 267353.0 267353.0 267353.0 267353.0 [0.9933448  0.00464595]
712 id_35_count 149464.0 149464.0 149464.0 149464.0 [0.9933448  0.00464595]
711 id_34_count 132185.0 17377.0 132185.0 17377.0 [0.9933448  0.00464595]
710 id_33_count 8127.0 6556.0 8127.0 6556.0 [0.9931946  0.00464595]
709 id_32_count 104040.0 44077.0 104040.0 44077.0 [0.99311996 0.00464595]
708 id_31_count 926.0 23650.0 926.0 23655.0 [0.99476165 0.00464595]
707 id_30_count 42170.0 711.0 42170.0 711.0 [0.99547523 0.00464595]
706 id_29_count 149264.0 149264.0 149264.0 149264.0 [0.99547523 0.00

586 ukey_C1scount nan nan nan nan [0.9223579  0.00464595]
585 ukey_D15count nan nan nan nan [0.9223579  0.00464595]
584 ukey_D1count 1.0 nan 1.0 nan [0.9219066  0.00464595]
583 ukey_C13count 1.0 nan 1.0 nan [0.88705736 0.00464595]
582 ukey_C7count 1.0 1.0 1.0 1.0 [0.88705736 0.00464595]
581 ukey_C5count 1.0 1.0 1.0 1.0 [0.88705736 0.00464595]
580 ukey_C3count 1.0 1.0 1.0 1.0 [0.88705736 0.00464595]
579 TransactionAmt_ukeystd nan nan nan nan [0.88705736 0.00464595]
578 TransactionAmt_ukeysum 225.0 20.0 225.0 20.0 [0.87762994 0.00464595]
577 TransactionAmt_ukeycount 1 1 1 1 [0.87762994 0.00464595]
576 TransactionDT2_uid2_TransactionAmt_V127scount nan nan nan nan [0.87762994 0.00464595]
575 TransactionDT2_uid2_TransactionAmt_V310scount nan nan nan nan [0.87762994 0.00464595]
574 TransactionDT2_uid2_TransactionAmt_V307scount nan nan nan nan [0.87762994 0.00464595]
573 TransactionDT2_uid2_TransactionAmt_D16scount nan nan nan nan [0.87762994 0.00464595]
572 TransactionDT2_uid2_TransactionAmt

478 en_card1card2_count 0.0 0.0 0.0 0.0 [0.7415385  0.00464595]
477 en_card2addr1_count 4.934 4.277 4.933088184379908 4.276295995965456 [0.7415535  0.00464595]
476 card2_addr1_amtcount 174.9 155.5 174.8476857142856 155.55277153558046 [0.7415535  0.00464595]
475 card1_card2_amtcount 232.5 84.0 232.44558464223377 84.0 [0.747486   0.00464595]
474 card1_amtcount 232.2 84.0 232.30195121951212 84.0 [0.736553   0.00464595]
473 TransactionAmt_decimal 0 0 0 0 [0.736553   0.00464595]
472 weekday 1 1 1 1 [0.736553   0.00464595]
471 hour 1 1 1 1 [0.736553   0.00464595]
470 w2v_addr1_23 0.0235 -0.008545 0.023500603345490197 -0.00854505815549937 [0.73178387 0.00464595]
469 w2v_addr1_22 -0.0666 -0.01741 -0.06658854561759689 -0.017410338515297597 [0.7158106  0.00464595]
468 w2v_addr1_21 0.01979 0.02142 0.01979463506834131 0.021420235752430804 [0.7158106  0.00464595]
467 w2v_addr1_20 -0.01854 0.00584 -0.018533120600848673 0.005839102903062731 [0.7158106  0.00464595]
466 w2v_addr1_19 0.02744 -0.02328 0.

391 id_31_raw 0 0 0 0 [0.74090123 0.00464595]
390 V339 0.0 0.0 0.0 0.0 [0.74090123 0.00464595]
389 V338 0.0 0.0 0.0 0.0 [0.74090123 0.00464595]
388 V337 0.0 0.0 0.0 0.0 [0.74090123 0.00464595]
387 V336 0.0 0.0 0.0 0.0 [0.74090123 0.00464595]
386 V335 0.0 20.0 0.0 20.0 [0.74090123 0.00464595]
385 V334 0.0 0.0 0.0 0.0 [0.74090123 0.00464595]
384 V333 0.0 0.0 0.0 0.0 [0.74090123 0.00464595]
383 V332 0.0 20.0 0.0 20.0 [0.74214923 0.00464595]
382 V331 0.0 0.0 0.0 0.0 [0.74214923 0.00464595]
381 V330 0.0 0.0 0.0 0.0 [0.74214923 0.00464595]
380 V329 0.0 0.0 0.0 0.0 [0.74214923 0.00464595]
379 V328 0.0 0.0 0.0 0.0 [0.74214923 0.00464595]
378 V327 0.0 0.0 0.0 0.0 [0.74214923 0.00464595]
377 V326 0.0 1.0 0.0 1.0 [0.7419958  0.00464595]
376 V325 0.0 0.0 0.0 0.0 [0.7419958  0.00464595]
375 V324 0.0 0.0 0.0 0.0 [0.7419958  0.00464595]
374 V323 0.0 1.0 0.0 1.0 [0.7419958  0.00464595]
373 V322 0.0 0.0 0.0 0.0 [0.7419958  0.00464595]
372 V321 0.0 0.0 0.0 0.0 [0.7419958  0.00464595]
371 V320 0.0 0.0 0.

223 V172 0.0 0.0 0.0 0.0 [0.1087286  0.00464595]
222 V171 4.0 1.0 4.0 1.0 [0.10787867 0.00464595]
221 V170 4.0 1.0 4.0 1.0 [0.10640191 0.00464595]
220 V169 0.0 0.0 0.0 0.0 [0.10640191 0.00464595]
219 V168 3.0 0.0 3.0 0.0 [0.10824413 0.00464595]
218 V167 3.0 0.0 3.0 0.0 [0.10824413 0.00464595]
217 V166 475.0 0.0 475.0 0.0 [0.10824413 0.00464595]
216 V165 250.0 0.0 250.0 0.0 [0.10692805 0.00464595]
215 V164 250.0 0.0 250.0 0.0 [0.10101806 0.00464595]
214 V163 0.0 0.0 0.0 0.0 [0.10101806 0.00464595]
213 V162 0.0 0.0 0.0 0.0 [0.10101806 0.00464595]
212 V161 0.0 0.0 0.0 0.0 [0.10101806 0.00464595]
211 V160 0.0 40.0 0.0 40.0 [0.09939989 0.00464595]
210 V159 0.0 0.0 0.0 0.0 [0.09939989 0.00464595]
209 V158 0.0 0.0 0.0 0.0 [0.09939989 0.00464595]
208 V157 0.0 0.0 0.0 0.0 [0.09939989 0.00464595]
207 V156 0.0 0.0 0.0 0.0 [0.09939989 0.00464595]
206 V155 0.0 0.0 0.0 0.0 [0.09939989 0.00464595]
205 V154 0.0 0.0 0.0 0.0 [0.09939989 0.00464595]
204 V153 0.0 0.0 0.0 0.0 [0.09939989 0.00464595]
203 V1

52 V1 nan nan nan nan [0.06076464 0.00464595]
51 M9 2 2 nan nan [0.06076464 0.00464595]
50 M8 2 2 nan nan [0.06076464 0.00464595]
49 M7 2 2 nan nan [0.06076464 0.00464595]
48 M6 2 2 nan nan [0.06076464 0.00464595]
47 M5 2 2 nan nan [0.06076464 0.00464595]
46 M4 3 3 nan nan [0.06076464 0.00464595]
45 M3 2 2 nan nan [0.06076464 0.00464595]
44 M2 2 2 nan nan [0.06076464 0.00464595]
43 M1 2 2 nan nan [0.06076464 0.00464595]
42 D15 nan 280.0 nan 280.0 [0.06326344 0.00464595]
41 D14 386.0 nan 386.0 nan [0.06255651 0.00464595]
40 D13 nan nan nan nan [0.06255651 0.00464595]
39 D12 nan nan nan nan [0.06255651 0.00464595]
38 D11 nan nan nan nan [0.06255651 0.00464595]
37 D10 nan nan nan nan [0.06255651 0.00464595]
36 D9 0.04166 0.04166 0.04166600108146668 0.04166600108146668 [0.06255651 0.00464595]
35 D8 1.042 17.05 1.041666030883789 17.04166603088379 [0.06089405 0.00464595]
34 D7 nan 16.0 nan 16.0 [0.06089405 0.00464595]
33 D6 nan 55.0 nan 55.0 [0.06089405 0.00464595]
32 D5 nan nan nan nan [0.0

In [62]:
features = [x for x in X_train.columns if (x not in col_del_ukey and  not x.startswith("w2v") and x not in ['TransactionDT'] )]
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
folds = 3
kf = KFold(n_splits = folds, shuffle = False, random_state=seed+1)
y_preds11 = np.zeros(X_test.shape[0])
y_oof = np.zeros(X_train.shape[0])
i = 0
for tr_idx, val_idx in kf.split(X_train, y_train):
    print(val_idx)
    i+=1
    clf = xgb.XGBClassifier(
        n_estimators=700,
        max_depth=9,
        learning_rate=0.03,
        subsample=0.9,
        colsample_bytree=0.9,
        tree_method='gpu_hist'
    )
    
    
    X_tr, X_vl = X_train[features].iloc[tr_idx, :], X_train[features].iloc[val_idx, :]
    y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
    clf.fit(X_tr, y_tr)
    y_pred_train = clf.predict_proba(X_vl)[:,1]
    y_oof[val_idx] = y_pred_train
    print('ROC AUC {}'.format(roc_auc_score(y_vl, y_pred_train)))
    
    temp = clf.predict_proba(X_test[features])[:,1] 
    y_preds11+= temp/ folds
    X_test_pred['pred_2'] += temp / folds
    if debug:    
        print("debug:",roc_auc_score(y_test, temp),log_loss(y_test, temp),metric(y_test, temp))   
    del X_tr,X_vl
        

if debug:    
    print("debug:",roc_auc_score(y_test, y_preds11),log_loss(y_test, y_preds11),metric(y_test, y_preds11))  
    print("debug:",roc_auc_score(y_test, y_preds11*0.5 + y_preds*0.5),log_loss(y_test, y_preds11*0.5 + y_preds*0.5),metric(y_test, y_preds11*0.5 + y_preds*0.5))

X_train_pred['pred_2'] = y_oof
   

[     0      1      2 ... 196844 196845 196846]
ROC AUC 0.9282657050882961
[196847 196848 196849 ... 393691 393692 393693]
ROC AUC 0.9421732287231162
[393694 393695 393696 ... 590537 590538 590539]
ROC AUC 0.9276788359520768


In [63]:
if debug:    
    print("debug:",roc_auc_score(y_test, y_preds11),log_loss(y_test, y_preds11),metric(y_test, y_preds11))  
    print("debug:",roc_auc_score(y_test, y_preds11*0.5 + y_preds*0.5),log_loss(y_test, y_preds11*0.5 + y_preds*0.5),metric(y_test, y_preds11*0.5 + y_preds*0.5))

from sklearn.feature_extraction import DictVectorizer
# print(eli5.explain_prediction_xgboost(clf,X_test.iloc[0,:], vec = vec))
eli5.show_weights(clf)
# print(res.data)
# eli5.explain_prediction(clf,X_test.iloc[10,:])
# eli5.show_prediction(clf, X_train.iloc[10:11,:],show_feature_values=True)


Weight,Feature
0.0892,V258
0.0335,V70
0.0200,V257
0.0187,id_12_count
0.0173,V189
0.0119,C1_count
0.0114,V294
0.0110,V91
0.0100,TransactionDT2_uid2_TransactionAmt_C13count
0.0095,C4


In [64]:
import lightgbm as lgb
# feat_col = [x for x in X_train.columns if x in ['stationID','hour', 'min', 'minandhour','num_neighboor', 'station_time0', 'station_time1',
#        'station_time2', 'station_time3', 'station_pos', 'station_pos2',  'datediff',
#        'station_time_diff_0', 'station_time_diff_1', 'station_time_diff_2',
#        'station_time_diff_3', 'th', 'tl', 'cloud', 'rain1','line0','line0b','line1','line2',
#        'rain2', 'cloud2', 'cloud3', 'sun', 'weekday', 'weekend','type','railway','busstation'] or x.startswith("svd") or x.startswith("w2v") or x.startswith("poi")]
features = [x for x in X_train.columns if x not in ['TransactionDT'] ]  
cate = [x for x in features if (x == 'ProductCD' or  x.startswith("addr") or x.startswith("card")  or 
                                       x.endswith("domain") or
                                       x in ['id_02','id_27','id_28','id_29','id_30','id_31','id_32','id_33','id_23',
                                            'id_30_raw', 'id_31_raw', 'DeviceInfo_raw']) and not x.endswith("count")
#         and x not in train_f6.columns
       ] #
print(cate)
params = {'application': 'binary',
          'boosting': 'gbdt',
          'metric': 'binary_logloss,auc',
          'max_depth': 16,
          'learning_rate': 0.05,
          'bagging_fraction': 0.9,
          'feature_fraction': 0.9,
          'verbosity': -1,
          'lambda_l1': 0.5,
          'lambda_l2': 0.05,
          'num_leaves': 500,
          'min_child_weight': 5,
          'min_data_in_leaf': 25,
          "num_threads":4,
          'metric_freq':10,
          'data_random_seed': 17}
print(params)
early_stop = 500
verbose_eval = 30
num_rounds = 800
# 
folds = 3
kf = KFold(n_splits = folds, shuffle = False, random_state=seed + 2)
y_preds2 = np.zeros(X_test.shape[0])
y_oof = np.zeros(X_train.shape[0])
feature_importance_df = pd.DataFrame()
i = 0
for tr_idx, val_idx in kf.split(X_train, y_train):
# for tr_idx, val_idx in folds_index:
    print(val_idx)
    X_tr, X_vl = X_train[features].iloc[tr_idx, :], X_train[features].iloc[val_idx, :]
    y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
    d_train = lgb.Dataset(X_tr, label=y_tr,categorical_feature = cate)
    d_valid = lgb.Dataset(X_vl, label=y_vl,categorical_feature = cate)
    watchlist = [d_valid]
    if debug:
        d_test = lgb.Dataset(X_test[features], label=y_test,categorical_feature = cate)
        watchlist.append(d_test)
    
    
    model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=num_rounds,
                      valid_sets=watchlist,
                      verbose_eval=verbose_eval)
        
    y_pred_train = model.predict(X_vl)
    y_oof[val_idx] = y_pred_train
    print('ROC AUC {}'.format(roc_auc_score(y_vl, y_pred_train)))
    
    temp = model.predict(X_test[features])
    y_preds2+= temp / folds
    X_test_pred['pred_3'] += temp / folds
    
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = X_tr.columns
    fold_importance_df["importance"] = model.feature_importance()
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    if debug:    
        print("debug:",roc_auc_score(y_test, temp),log_loss(y_test, temp),metric(y_test, temp))  
    i+=1

if debug:    
    print("debug:",roc_auc_score(y_test, y_preds2),log_loss(y_test, y_preds2),metric(y_test, y_preds2))  
    print("debug:",roc_auc_score(y_test, (y_preds + y_preds2)*0.5),log_loss(y_test, (y_preds + y_preds2)*0.5),metric(y_test, (y_preds + y_preds2)*0.5))  
    print("debug:",roc_auc_score(y_test, (y_preds11 + y_preds2)*0.5),log_loss(y_test, (y_preds11 + y_preds2)*0.5),metric(y_test, (y_preds11 + y_preds2)*0.5))  

X_train_pred['pred_3'] = y_oof


['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'id_31_raw', 'id_30_raw', 'DeviceInfo_raw', 'id_02', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33']
{'application': 'binary', 'boosting': 'gbdt', 'metric': 'binary_logloss,auc', 'max_depth': 16, 'learning_rate': 0.05, 'bagging_fraction': 0.9, 'feature_fraction': 0.9, 'verbosity': -1, 'lambda_l1': 0.5, 'lambda_l2': 0.05, 'num_leaves': 500, 'min_child_weight': 5, 'min_data_in_leaf': 25, 'num_threads': 4, 'metric_freq': 10, 'data_random_seed': 17}
[     0      1      2 ... 196844 196845 196846]


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[30]	valid_0's auc: 0.897884	valid_0's binary_logloss: 0.0835119
[60]	valid_0's auc: 0.906067	valid_0's binary_logloss: 0.0767088
[90]	valid_0's auc: 0.906786	valid_0's binary_logloss: 0.0752524
[120]	valid_0's auc: 0.907401	valid_0's binary_logloss: 0.0749269
[150]	valid_0's auc: 0.909746	valid_0's binary_logloss: 0.0746929
[180]	valid_0's auc: 0.911556	valid_0's binary_logloss: 0.0747831
[210]	valid_0's auc: 0.912193	valid_0's binary_logloss: 0.0751449
[240]	valid_0's auc: 0.912096	valid_0's binary_logloss: 0.0758785
[270]	valid_0's auc: 0.912699	valid_0's binary_logloss: 0.0765622
[300]	valid_0's auc: 0.91311	valid_0's binary_logloss: 0.077248
[330]	valid_0's auc: 0.913758	valid_0's binary_logloss: 0.0779647
[360]	valid_0's auc: 0.914542	valid_0's binary_logloss: 0.0785265
[390]	valid_0's auc: 0.914885	valid_0's binary_logloss: 0.0792845
[420]	valid_0's auc: 0.915374	valid_0's binary_logloss: 0.0799706
[450]	valid_0's auc: 0.915453	valid_0's binary_logloss: 0.0806792
[480]	valid_0's

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[30]	valid_0's auc: 0.910997	valid_0's binary_logloss: 0.100143
[60]	valid_0's auc: 0.92249	valid_0's binary_logloss: 0.0919064
[90]	valid_0's auc: 0.928005	valid_0's binary_logloss: 0.0899225
[120]	valid_0's auc: 0.928925	valid_0's binary_logloss: 0.0902436
[150]	valid_0's auc: 0.928315	valid_0's binary_logloss: 0.091321
[180]	valid_0's auc: 0.928182	valid_0's binary_logloss: 0.0922564
[210]	valid_0's auc: 0.928078	valid_0's binary_logloss: 0.0933911
[240]	valid_0's auc: 0.928539	valid_0's binary_logloss: 0.0944127
[270]	valid_0's auc: 0.928557	valid_0's binary_logloss: 0.0956381
[300]	valid_0's auc: 0.928466	valid_0's binary_logloss: 0.0970091
[330]	valid_0's auc: 0.928852	valid_0's binary_logloss: 0.09821
[360]	valid_0's auc: 0.928916	valid_0's binary_logloss: 0.0994676
[390]	valid_0's auc: 0.929377	valid_0's binary_logloss: 0.100526
[420]	valid_0's auc: 0.929475	valid_0's binary_logloss: 0.101586
[450]	valid_0's auc: 0.929595	valid_0's binary_logloss: 0.102562
[480]	valid_0's auc: 

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[30]	valid_0's auc: 0.904702	valid_0's binary_logloss: 0.095022
[60]	valid_0's auc: 0.914109	valid_0's binary_logloss: 0.0870046
[90]	valid_0's auc: 0.915873	valid_0's binary_logloss: 0.0857594
[120]	valid_0's auc: 0.917435	valid_0's binary_logloss: 0.0856098
[150]	valid_0's auc: 0.917879	valid_0's binary_logloss: 0.086163
[180]	valid_0's auc: 0.918447	valid_0's binary_logloss: 0.0866656
[210]	valid_0's auc: 0.918872	valid_0's binary_logloss: 0.0874512
[240]	valid_0's auc: 0.919601	valid_0's binary_logloss: 0.0882459
[270]	valid_0's auc: 0.920237	valid_0's binary_logloss: 0.0892303
[300]	valid_0's auc: 0.920661	valid_0's binary_logloss: 0.0902209
[330]	valid_0's auc: 0.920865	valid_0's binary_logloss: 0.0911935
[360]	valid_0's auc: 0.920701	valid_0's binary_logloss: 0.0923508
[390]	valid_0's auc: 0.920717	valid_0's binary_logloss: 0.093299
[420]	valid_0's auc: 0.921288	valid_0's binary_logloss: 0.0940241
[450]	valid_0's auc: 0.921519	valid_0's binary_logloss: 0.0950204
[480]	valid_0's 

In [65]:
if debug:
    X_train_pred.to_csv('../input/xgb_33_train_debug.csv')
    X_test_pred.to_csv('../input/xgb_33_test_debug.csv')
else:
    X_train_pred.to_csv('../input/xgb_33_train.csv')
    X_test_pred.to_csv('../input/xgb_33_test.csv')
    
del X_test_pred,X_train_pred 